# 新增患者数量作为输入变量

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [2]:
path = 'C:/Users/new/Desktop/疫情演化系统/数据/Trajectory_Geocoding.csv'
data = pd.read_csv(path)

In [3]:
data.drop(['原始文本', '来源', '风险等级', '存疑'], axis=1, inplace=True) # 去掉不需要的列

In [4]:
data.head()

,病例号,起始日期,起始时间,结束日期,结束时间,场所,经度,纬度,完整地址,所在区,街道,township
0,1,2021/12/23,14:01:00,NaN,NaN,地铁6号线南翠屏站,117.158640,39.079964,天津市南开区体育中心街道南翠屏(地铁站)南翠屏公园,南开区,宾水西道,体育中心街道
1,1,2021/12/23,14:11:00,NaN,NaN,地铁5号线肿瘤医院站,117.183701,39.080221,天津市河西区天塔街道天津市肿瘤医院,河西区,环湖西路,天塔街道
2,1,2021/12/23,14:21:00,NaN,NaN,地铁1号线下瓦房站,117.222795,39.140884,天津市河东区春华街道远洋天地新天地家园,河东区,华捷道,春华街道
3,1,2021/12/23,14:41:00,NaN,NaN,地铁1号线财经大学站,117.278442,39.061219,天津市河西区尖山街道天津财经大学-教学楼K座,河西区,珠江道,尖山街道
4,1,2021/12/23,14:58:00,NaN,NaN,灰堆公交站,117.276909,39.071087,天津市河西区柳林街道大沽南路辅路,河西区,大沽南路,柳林街道


In [5]:
path = 'C:/Users/new/Desktop/疫情演化系统/数据/Patient_Geocoding.csv'
data_pat = pd.read_csv(path)

In [6]:
data_pat.drop(['来源1','来源2'], axis=1, inplace=True)

In [7]:
data_pat.head()

,患者序号,性别,年龄,职业,家庭住址,分型,完整地址,所在区,街道,township,经度,纬度
0,1,女,29,托辅机构工作人员,南开区华苑新城,轻型,天津市南开区华苑街道华苑新城,南开区,华苑路,华苑街道,117.143640,39.075703
1,2,女,10,咸水沽第七小学学生,津南区咸水沽镇,轻型,天津市津南区咸水沽镇咸水沽镇咸水沽镇人民政府,津南区,丰收路,咸水沽镇,117.395242,38.979148
2,3,男,10,咸水沽第七小学学生,津南区北闸口镇,轻型,天津市津南区北闸口镇政安里北闸口镇人民政府,津南区,北营路,北闸口镇,117.406017,38.950556
3,4,男,12,辛庄中学学生,津南区辛庄镇,轻型,天津市津南区辛庄镇龙汇公寓辛庄镇人民政府,津南区,平凡道,辛庄镇,117.350345,39.026690
4,5,男,13,辛庄中学学生,津南区辛庄镇,轻型,天津市津南区辛庄镇龙汇公寓辛庄镇人民政府,津南区,平凡道,辛庄镇,117.350345,39.026690


In [8]:
path = 'C:/Users/new/Desktop/疫情演化系统/数据/train_township.csv'
train = pd.read_csv(path)

In [9]:
train["患者数量"] = 0

In [10]:
for index,row in train.iterrows():
    train.loc[index, '日期'] = datetime.strptime(row['日期'], "%Y/%m/%d")

In [11]:
train

,日期,镇区,经停病例数,患者数量,经度,纬度
0,2021-12-23 00:00:00,东马圈镇,0,0,116.844191,39.444862
1,2021-12-24 00:00:00,东马圈镇,0,0,116.844191,39.444862
2,2021-12-25 00:00:00,东马圈镇,0,0,116.844191,39.444862
3,2021-12-26 00:00:00,东马圈镇,0,0,116.844191,39.444862
4,2021-12-27 00:00:00,东马圈镇,0,0,116.844191,39.444862
...,...,...,...,...,...,...
6227,2022-01-28 00:00:00,北仓镇,0,0,117.117081,39.233295
6228,2022-01-29 00:00:00,北仓镇,0,0,117.117081,39.233295
6229,2022-01-30 00:00:00,北仓镇,0,0,117.117081,39.233295
6230,2022-01-31 00:00:00,北仓镇,0,0,117.117081,39.233295


In [12]:
startdatetime = datetime.strptime('2022-01-07', "%Y-%m-%d")
for index,row in data_pat.iterrows():
    pat_id = row['患者序号']
    township = row['township']
    # 找出患者检测出阳性的时间
    rows = data[(data['病例号'] == pat_id)]
    d = rows.iloc[len(rows)-1, 1]
    if d is not np.NaN:
        d = datetime.strptime(d, "%Y/%m/%d")
        if startdatetime < d:
            startdatetime = d
    rows_index = train[(train['日期'] == startdatetime) & (train['镇区'] == township)].index.tolist()
    if rows_index:
        for r in rows_index:
            # 这里假设一个患者有14天的影响
            for time_lag in range(14):
                if train.loc[r+time_lag, '镇区'] == township:
                    train.loc[r+time_lag, '患者数量'] += 1

In [13]:
train

,日期,镇区,经停病例数,患者数量,经度,纬度
0,2021-12-23 00:00:00,东马圈镇,0,0,116.844191,39.444862
1,2021-12-24 00:00:00,东马圈镇,0,0,116.844191,39.444862
2,2021-12-25 00:00:00,东马圈镇,0,0,116.844191,39.444862
3,2021-12-26 00:00:00,东马圈镇,0,0,116.844191,39.444862
4,2021-12-27 00:00:00,东马圈镇,0,0,116.844191,39.444862
...,...,...,...,...,...,...
6227,2022-01-28 00:00:00,北仓镇,0,0,117.117081,39.233295
6228,2022-01-29 00:00:00,北仓镇,0,0,117.117081,39.233295
6229,2022-01-30 00:00:00,北仓镇,0,0,117.117081,39.233295
6230,2022-01-31 00:00:00,北仓镇,0,0,117.117081,39.233295


In [14]:
output_path='C:/Users/new/Desktop/train_township.csv'
train.to_csv(output_path, sep=',', index=False, header=True, encoding='utf_8_sig')